## Data processing for Moored and DP Platforms

By the end of this tutorial, you will be able to: 

### Handle strain data from the moored platform
- 1.1 read qualisys data and synchronize strain measurements
- 1.2 calibrate strain guage readings
- 1.3 plot displacement and tension for the moored platform

### Handle thrust data from the DP platform (Smarty)
- 2.1 apply the equivalent pipeline for the DP platform


### Data processing pipeline for moored platform

Figure below explains the data processing pipeline for moored platform.

![Data processing schematic for moored platform](pipeline_diagram_smarty.png)

### 1.1 Synchronize qualisys and strain measurements

In [ ]:
# import libraries
from analysis import *
# make sure you use the latest version as there is a small modification!!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
filename = '../Qualisys/run0008_0009_6D.tsv'
#filename = '../Qualisys/run0011_6D.tsv'
moored_qualisys_df = read_moored(filename)
list(moored_qualisys_df)

Plot the displacement in the wave direction (remember this was q_3 from last week)

In [ ]:
plt.plot(moored_qualisys_df['time'], moored_qualisys_df['q_3'],'b.',markersize=1)

Remember qualisys reports position as __zero__ when it can't detect the the platform. We can filter these using "query".

In [ ]:
# select all points with non-zero displacement
moored_qualisys_df = moored_qualisys_df.query('q_3!=0') 

plt.plot(moored_qualisys_df['time'], moored_qualisys_df['q_3'],'b.',markersize=1)

We are only interested in the data once the waves arrive at the platform. We can search for this when using a threshold search function.

In [ ]:
def detect_start_index(df, parameter, threshold, delay=0):
    
    for index, row in df.iterrows():
        if row['time'] > delay:
            break
    
    #store start index and start value
    start_index = index
    start_value = df[parameter][index]
   
    #return index when parameter next deviates by more than the threshold
    for index, row in df.iterrows():
        if row['time'] > delay:
            if df[parameter][index] > start_value+threshold or df[parameter][index] < start_value-threshold:
                break
    
    #plot to see what value has been found
    plt.plot(df['time'], df[parameter],'b.',markersize=1)
    plt.plot(df['time'][index], df[parameter][index],'ro',markersize=10)    
    return index

# search for start of second run (hint ~approx 550s)
start_qualisys=detect_start_index(moored_qualisys_df,'q_3',0.05, 550)
print(moored_qualisys_df['time'][start_qualisys])

The next step is to trim the dataset and measure time and displacement from this starting index

In [ ]:
moored_qualisys_df_truncated = moored_qualisys_df

moored_qualisys_df_truncated['time'] = moored_qualisys_df['time'] - moored_qualisys_df['time'][start_qualisys]
moored_qualisys_df_truncated['q_3'] = moored_qualisys_df['q_3'] - moored_qualisys_df['q_3'][start_qualisys]

#use query to just capture the increasing displasement when the mooring line is being stretched
moored_qualisys_df_truncated = moored_qualisys_df_truncated.query('0<time<60')

#plot
plt.plot(moored_qualisys_df_truncated['time'], moored_qualisys_df_truncated['q_3'],'b.')

Now we repeat the same procedure for the straing measurements

In [ ]:
moored_strain_df = pd.read_csv('../MooredStrain/run8_9/strain.csv')
list(moored_strain_df)

In [ ]:
# epoch?
print(moored_strain_df['epoch_time'][0])

# create a field for elapsed 'time' from start of acquisition 
moored_strain_df['time']=moored_strain_df['epoch_time']-moored_strain_df['epoch_time'][0]
plt.plot(moored_strain_df['time'], moored_strain_df[' strain'])

Detect the start of the 2nd run as for qualisys

In [ ]:
start_strain = detect_start_index(moored_strain_df, ' strain', 3, 350)
print (moored_strain_df['time'][start_strain])

Trim data to get the bit we are interested in and measure strain relative to initial condition

In [ ]:
moored_strain_df_truncated = moored_strain_df

moored_strain_df_truncated['time'] = moored_strain_df['time'] - moored_strain_df['time'][start_strain]
moored_strain_df_truncated[' strain'] = moored_strain_df[' strain'] - moored_strain_df[' strain'][start_strain]

# use query as before 
moored_strain_df_truncated = moored_strain_df_truncated.query('0<time<60')

#plot
plt.plot(moored_strain_df_truncated['time'], moored_strain_df_truncated[' strain'])

To synchronize datasets, first check sampling frequencies.

In [ ]:
# strain
plt.plot (moored_strain_df_truncated.time.diff(),'r.',markersize=2)
moored_strain_frequency = 1/moored_strain_df_truncated.time.diff().mean()
print ("Mean frequency: ", moored_strain_frequency, "Hz")

What do you notice about the qualisys plot? is mean a good representation?

In [ ]:
# qualisys
plt.plot (moored_qualisys_df_truncated.time.diff(),'k.',markersize=2)
moored_qualisys_frequency = 1/moored_qualisys_df_truncated.time.diff().mean()
print ("Meanfrequency: ", moored_qualisys_frequency, "Hz")

#alternative to mean that is more representitive?
print ("Mode frequency: ", 1/moored_qualisys_df.time.diff().mode(), "Hz")
plt.ylim(0,0.05)

We want to match to a common frequency. What would make sense?

In [ ]:
moored_qualisys_frequency = 10 # Hz
#resampling from strain frequency to qualisys frequency
moored_strain_df_truncated = resample_evenly(moored_strain_df_truncated, moored_qualisys_frequency)

#resampling qualisys to fill missing values
moored_qualisys_df_truncated = resample_evenly(moored_qualisys_df_truncated, moored_qualisys_frequency)

Check!

In [ ]:
moored_strain_frequency = 1/moored_strain_df_truncated.time.diff().mean()
print ("Mean sampling period: ", 1/moored_strain_frequency, "seconds")
print ("Mean sampling frequency: ", moored_strain_frequency, "Hz")

moored_qualisys_frequency = 1/moored_qualisys_df_truncated.time.diff().mean()
print ("Mean sampling period: ", 1/moored_qualisys_frequency, "seconds")
print ("Mean sampling frequency: ", moored_qualisys_frequency, "Hz")


### 1.2 Calibrate strain gauge measurements

Strain gauge can be calibrated to tension (N) using (calibratrion.csv).

In [ ]:
# read in the calibration data
moored_calibration = pd.read_csv('../MooredStrain/calibration.csv')
list(moored_calibration)

Plot mass (kg) vs force (N). Is this what you would expect?

In [ ]:
gravity = 9.81
moored_calibration['Tension (N)']  = moored_calibration['mass (kg)']*gravity
plt.plot(moored_calibration[' AD value'], moored_calibration['Tension (N)'],'ro')
plt.xlabel("AD")
plt.ylabel("Tension [N]")

Fit a line of best fit.

In [ ]:
order = 1
[slope,intercept] = np.polyfit(moored_calibration[' AD value'], moored_calibration['Tension (N)'], order)
print ("Slope: ",slope, "\tIntercept: ", intercept)

In [ ]:
def apply_calibration(data_frame, column_to_calibrate, slope,intercept,label_for_calibrated_data):
    data_frame[label_for_calibrated_data] = slope*data_frame[column_to_calibrate]+intercept
    return

# now use the subroutine to  apply the linear model to our calibration data
apply_calibration(moored_calibration,' AD value', slope, intercept, 'Calibrated Tension (N)')

plt.plot(moored_calibration[' AD value'], moored_calibration['Tension (N)'],'ro')
plt.plot(moored_calibration[' AD value'], moored_calibration['Calibrated Tension (N)'],'k-')
plt.xlabel("AD value")
plt.ylabel("Tension [N]")

In [ ]:
apply_calibration(moored_strain_df_truncated,[' strain'], slope, intercept, 'Tension (N)')

plt.plot(moored_strain_df_truncated['time'], moored_strain_df_truncated['Tension (N)'])
plt.grid(True)

__NOTE__: the initial value should be our steady state (no load) scenario. We must substract the Tension(N) value from the first sample

In [ ]:
tension = moored_strain_df_truncated['Tension (N)']-moored_strain_df_truncated['Tension (N)'][0]
displacement = abs(moored_qualisys_df_truncated['q_3'])

print(len(tension),len(displacement))

#  plot displacement vs strain
upper_lim=min(len(tension),len(displacement))
plt.plot(displacement[0:upper_lim], tension[0:upper_lim], 'ro', markersize=3)
plt.grid(True)

Does the plot make sense ? Do the values have a sensible order? Why is the  so noisey and what can you do to reduces it?


.

.

.

.

.

.

.

.

.

.

.

.
.
.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.







































# Task for later
Consider both X and Y displacement? Perform frequency analysis to extract mean load

# 2. Processing for DP (dynamic positioning) platform - [SMARTY]

Figure below explains the data processing pipeline for the DP platform.


![alt text](pipeline_diagram_smarty.png "Data processing schematic for moored platform")

In [ ]:
filename = '../Qualisys/run0008_0009_6D.tsv'
dp_qualisys_df = read_smarty(filename)

#filter
dp_qualisys_df=dp_qualisys_df.query('q_3!=0')

#plot
plt.plot(dp_qualisys_df['time'],dp_qualisys_df['q_3'])

In [ ]:
dp_thruster_df = pd.read_csv('../Smarty/run8_9/thrust_inertial.csv')
list(dp_thruster_df)

In [ ]:
# create a field for elapsed 'time' from start of acquisition 
dp_thruster_df['time']=dp_thruster_df['epoch_time']-dp_thruster_df['epoch_time'][0]

plt.plot(dp_thruster_df['time'],dp_thruster_df[' thrust_north (N)'])
plt.plot(dp_thruster_df['time'],dp_thruster_df[' thrust_east (N)'])
plt.legend()

Do the numbers make sense?

# Task for later

1. Detection of start time
2. Calculate distance from the target point (q_3, q_4 = [0, -3.5])
2. Resample to a uniform sampling frequency
3. Plot thruster force vs displacement